In [ ]:
import numpy as np

In [ ]:
import lsst.daf.butler as dafButler
from lsst.pipe.tasks.loadReferenceCatalog import LoadReferenceCatalogConfig, LoadReferenceCatalogTask
from lsst.utils import getPackageDir
import lsst.geom

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# https://developer.lsst.io/usdf/storage.html#butler-access
repo = '/sdf/group/rubin/repo/main_20210215/'

In [ ]:
config= os.path.join(repo, 'butler.yaml')
butler = dafButler.Butler(config=config)
registry = butler.registry

In [ ]:
collection = 'refcats'

In [ ]:
registry.getCollectionSummary(collection).dataset_types.names

In [ ]:
refDataset = 'gaia_dr3'

In [ ]:
# We can also load the refcat with a spatial query
config = LoadReferenceCatalogConfig()
#config.refObjLoader.ref_dataset_name = refDataset

#config.refObjLoader.load(os.path.join(getPackageDir('obs_lsst'),
#                                      'config',
#                                      'filterMap.py'))
config.refObjLoader.anyFilterMapsToThis = 'phot_g_mean'
config.doApplyColorTerms = False

In [ ]:
dir(config.refObjLoader)

In [ ]:
config.refObjLoader.toDict()

In [ ]:
# There must be a more efficient spatial query to include here
# Perhaps https://github.com/rubin-dp0/tutorial-notebooks/blob/main/DP02_04b_Intermediate_Butler_Queries.ipynb 
# section 3.3.2 "User-defined spatial constraints on images"
datasetRefs = list(registry.queryDatasets(datasetType=refDataset, collections=collection).expanded())

In [ ]:
dataIds = [_.dataId for _ in datasetRefs]

In [ ]:
# Get DeferredDatasetHandles for reference catalog
refCats = [butler.getDeferred(refDataset, _, collections=['refcats']) for _ in dataIds]

In [ ]:
#loaderTask = LoadReferenceCatalogTask(config=config,
#                                      dataIds=dataIds,
#                                      refCats=refCats)

In [ ]:
loaderTask = LoadReferenceCatalogTask(config=config, name=refDataset, dataIds=dataIds, refCats=refCats)

In [ ]:
ra, dec = 180., -1.
center = lsst.geom.SpherePoint(ra,
                               dec,
                               lsst.geom.degrees)

In [ ]:
refCatSpatial = loaderTask.getSkyCircleCatalog(center,
                                               3.5*lsst.geom.degrees,
                                               ['i'])

In [ ]:
#refCatSpatial

In [ ]:
s = 10. - refCatSpatial['refMag'].flatten()

In [ ]:
np.max(s)

In [ ]:
selected = (refCatSpatial['refMag'].flatten() < 10.)
plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], s=2. * s[selected]**3, alpha=0.5, c=refCatSpatial['refMag'][selected])
plt.xlim(plt.xlim()[::-1])
plt.colorbar()

In [ ]:
s[selected]

In [ ]:
plt.figure()
plt.scatter(refCatSpatial['ra'], refCatSpatial['dec'], s=1)
plt.xlim(plt.xlim()[::-1])

# Transform from sky coordinates to camera coordinates

In [ ]:
from lsst.afw.cameraGeom import Camera

In [ ]:
#camera = butler.get('camera', instrument='LSSTCam')

In [ ]:
help(Camera)

In [ ]:
repo = '/sdf/group/rubin/repo/dc2_20210215'
collection = '2.2i/runs/DP0.1'

In [ ]:
butler = dafButler.Butler(repo, collections=collection)
camera = butler.get('camera', instrument='LSSTCam-imSim')

In [ ]:
camera

In [ ]:
dir(camera)

In [ ]:
for det in camera:
    center = det.getOrientation().getFpPosition()
    print(center)

In [ ]:
dir(det)

In [ ]:
det.getName()

In [ ]:
import lsst.afw.geom as afwGeom
import lsst.afw.cameraGeom as cameraGeom

In [ ]:
# https://lsstc.slack.com/archives/C2JPMCF5X/p1569266787124100
# https://community.lsst.org/t/constructing-a-wcs-from-camera-geometry/3039
def getWcsFromDetector(detector, boresight, rotation=0*lsst.geom.degrees, flipX=False):
    """Given a detector and (boresight, rotation), return that detector's WCS

    Parameters
    ----------
    detector : `lsst.afw.cameraGeom.Detector`
        A detector in a camera.
    boresight : `lsst.geom.SpherePoint`
       The boresight of the observation.
    rotation : `lsst.afw.geom.Angle`, optional
        The rotation angle of the camera.
        The rotation is "rotskypos", the angle of sky relative to camera
        coordinates (from North over East).
    flipX : `bool`, optional
        Flip the X axis?

    Returns
    -------
    wcs : `lsst::afw::geom::SkyWcs`
        The calculated WCS.
    """
    trans = detector.getTransform(detector.makeCameraSys(cameraGeom.PIXELS),
                                  detector.makeCameraSys(cameraGeom.FIELD_ANGLE))

    wcs = afwGeom.makeSkyWcs(trans, rotation, flipX, boresight)

    return wcs

In [ ]:
boresight = lsst.geom.SpherePoint(180., -1., units=lsst.geom.degrees)
rotation = 45.*lsst.geom.degrees
flipX = False

In [ ]:
trans = det.getTransform(det.makeCameraSys(cameraGeom.PIXELS), det.makeCameraSys(cameraGeom.FIELD_ANGLE))

In [ ]:
wcs = afwGeom.makeSkyWcs(trans, rotation, flipX, boresight)

In [ ]:
wcs

In [ ]:
wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)

In [ ]:
help(wcs.skyToPixelArray)

In [ ]:
x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)

In [ ]:
plt.figure()
plt.scatter(x, y)

In [ ]:
selected = det.getBBox().contains(x, y)

In [ ]:
plt.figure()
plt.scatter(x[~selected], y[~selected], s=1)
plt.scatter(x[selected], y[selected], s=1)

In [ ]:
det.getId()

In [ ]:
#detId = np.empty(len(refCatSpatial)).fill(-1.)
detId = np.zeros(len(refCatSpatial)) - 1.
for det in camera:
    if 'R22' not in det.getName():
        continue
    wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
    x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
    selected = det.getBBox().contains(x, y)
    detId[selected] = det.getId()
    print(det.getId(), det.getName(), np.sum(selected))

In [ ]:
def getDetectorId(refCatSpatial, camera, boresight, rotation, flipX):
    detId = np.zeros(len(refCatSpatial)) - 1.
    for det in camera:
        
        # Emulate ComCam by considering only the central raft
        if 'R22' not in det.getName():
            continue
            
        wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
        x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
        
        # Detector
        selected = det.getBBox().contains(x, y)
        detId[selected] = det.getId()
        #print(det.getId(), det.getName(), np.sum(selected))
        
        # Focal Plane Coordinates
        #xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
        #x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])

    #return detId, np.array(x_fp), np.array(y_fp)
    #return detId, x_fp, y_fp
    return detId

In [ ]:
def getFpCoordinates(refCatSpatial, camera, boresight, rotation, flipX):
    det = camera.get('R22_S11')
    wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
    x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
    xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])
    return np.array(x_fp), np.array(y_fp)

In [ ]:
detId = getDetectorId(refCatSpatial, camera, boresight, rotation, flipX)
#detId, x_fp, y_fp = getDetectorId(refCatSpatial, camera, boresight, rotation, flipX)

In [ ]:
x_fp, y_fp = getFpCoordinates(refCatSpatial, camera, boresight, rotation, flipX)

In [ ]:
np.unique(detId)

In [ ]:
selected = (detId >= 0)

plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=detId[selected], s=1)
plt.colorbar()

In [ ]:
plt.figure()
plt.scatter(x_fp[selected], y_fp[selected], c=detId[selected], s=1)
plt.colorbar()

In [ ]:
detId = np.zeros(len(refCatSpatial)) - 1
print(detId.shape)
print(detId)

In [ ]:
detId = np.empty(len(refCatSpatial))
print(detId.shape)
print(detId)

# Set of dithers

In [ ]:
n = 20
ra_boresight_array = 180. + np.random.uniform(-0.2, 0.2, size=n)
dec_boresight_array = -1. + np.random.uniform(-0.2, 0.2, size=n)
rotation_array = np.random.uniform(-180., 180., size=n)

In [ ]:
plt.figure()
plt.scatter(ra_boresight_array, dec_boresight_array, c=rotation_array)
plt.colorbar(label='Rotation (deg)')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
plt.suptitle('Telescope Boresight Pointings and Camera Rotation Angles')

In [ ]:
detId_array = []
x_fp_array = []
y_fp_array = []
for visit_index in range(0, n):
    print(visit_index)
    boresight_visit = lsst.geom.SpherePoint(ra_boresight_array[visit_index], dec_boresight_array[visit_index], units=lsst.geom.degrees)
    rotation_visit = rotation_array[visit_index]*lsst.geom.degrees
    detId = getDetectorId(refCatSpatial, camera, boresight_visit, rotation_visit, flipX)
    detId_array.append(detId)
    x_fp, y_fp = getFpCoordinates(refCatSpatial, camera, boresight_visit, rotation_visit, flipX)
    x_fp_array.append(x_fp)
    y_fp_array.append(y_fp)

In [ ]:
detId_array = np.array(detId_array)
x_fp_array = np.array(x_fp_array)
y_fp_array = np.array(y_fp_array)

In [ ]:
detId_array.shape

In [ ]:
counts = np.sum(detId_array >= 0, axis=0)

In [ ]:
selected = (counts > 0)
plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=counts[selected], s=1)
plt.colorbar(label='Number of Measurements')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')

In [ ]:
plt.figure()
plt.plot(np.sort(counts[selected])[::-1])
plt.xlabel('Number of Reference Stars')
plt.ylabel('Number of Measurements')

In [ ]:
# Number of unique detectors that each star appeared on

In [ ]:
unique_det = np.array([len(np.unique(_[_ >= 0])) for _ in detId_array.T])

In [ ]:
selected = (counts >= 1)
plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=unique_det[selected], s=1)
plt.colorbar(label='Number of Unique Detectors')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')

In [ ]:
plt.figure()
plt.plot(np.sort(unique_det[selected])[::-1])
plt.xlabel('Number of Reference Stars')
plt.ylabel('Number of Unique Detectors')

In [ ]:
# Number of repeated visits in focal plane coordinates
# Need to define some minimum number of measurements per star

In [ ]:
counts_array = np.tile(counts, detId_array.shape[0]).reshape(detId_array.shape)
unique_det_array = np.tile(unique_det, detId_array.shape[0]).reshape(detId_array.shape)

In [ ]:
plt.figure()
plt.scatter(x_fp_array[detId_array >= 0], y_fp_array[detId_array >= 0], c=unique_det_array[detId_array >= 0], s=1)
plt.colorbar(label='Number of Unique Detectors')
plt.xlabel('Focal Plane x (mm)')
plt.ylabel('Focal Plane y (mm)')

In [ ]:
plt.figure()
plt.scatter(x_fp_array[detId_array >= 0], y_fp_array[detId_array >= 0], c=counts_array[detId_array >= 0], s=1)
plt.colorbar(label='Number of Measurements')
plt.xlabel('Focal Plane x (mm)')
plt.ylabel('Focal Plane y (mm)')

In [ ]:
plt.figure()
plt.scatter(x_fp_array[0][detId_array[0] >= 0], y_fp_array[0][detId_array[0] >= 0], s=1)

In [ ]:
#(x_fp_array.T)[counts >= 3].T

In [ ]:
detId_array.flatten()

In [ ]:
x_fp_array.flatten()[(detId_array >= 0).flatten()].shape

In [ ]:
plt.figure()
plt.scatter(x_fp_array.flatten()[(detId_array >= 0).flatten()], y_fp_array.flatten()[(detId_array >= 0).flatten()], s=1)

In [ ]:
selection_counts = (counts >= 1)
selection_detected = (detId_array.T[selection_counts] >= 0)
x_fp_selected = x_fp_array.T[selection_counts][selection_detected]
y_fp_selected = y_fp_array.T[selection_counts][selection_detected]

In [ ]:
plt.figure()
plt.scatter(x_fp_selected, y_fp_selected, s=1)

In [ ]:
plt.figure()
plt.hexbin(x_fp_selected, y_fp_selected)

In [ ]:
dir(det)

In [ ]:
det.getCenter(cameraGeom.PIXELS)

In [ ]:
camera.get('R22_S11').getCenter(cameraGeom.PIXELS)

In [ ]:
# lsst.geom.Point2D(0., 0.)
det.getOrientation().getFpPosition()

In [ ]:
det.getOrientation().getReferencePoint()

In [ ]:
dir(det)

In [ ]:
help(det.getOrientation().makePixelFpTransform)

In [ ]:
xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)

In [ ]:
x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])

In [ ]:
plt.figure()
plt.scatter(x_fp, y_fp, s=1)

In [ ]:
det.getCenter(cameraGeom.FOCAL_PLANE)

In [ ]:
center = det.getCenter(cameraGeom.PIXELS)
x_center, y_center = center.x, center.y

In [ ]:
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
transform = LsstCameraTransforms(camera)

In [ ]:
help(transform.ccdPixelToFocalMm)

In [ ]:
transform.ccdPixelToFocalMm([0., 0.], [0., 0.], det.getName())

In [ ]:
xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])

In [ ]:
dir(camera)

In [ ]:
def getWcsFromCamera(camera, boresight, rotation=0*lsst.geom.degrees, flipX=False):
    """Given a detector and (boresight, rotation), return that detector's WCS

    Parameters
    ----------
    detector : `lsst.afw.cameraGeom.Detector`
        A detector in a camera.
    boresight : `lsst.geom.SpherePoint`
       The boresight of the observation.
    rotation : `lsst.afw.geom.Angle`, optional
        The rotation angle of the camera.
        The rotation is "rotskypos", the angle of sky relative to camera
        coordinates (from North over East).
    flipX : `bool`, optional
        Flip the X axis?

    Returns
    -------
    wcs : `lsst::afw::geom::SkyWcs`
        The calculated WCS.
    """
    trans = camera.getTransform(cameraGeom.PIXELS, cameraGeom.FIELD_ANGLE)

    wcs = afwGeom.makeSkyWcs(trans, rotation, flipX, boresight)

    return wcs

In [ ]:
wcs_camera = getWcsFromCamera(camera, boresight, rotation=rotation, flipX=flipX)

In [ ]:
help(camera.getTransform)